In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from pycaret.classification import *
from src.utils import *
from src.calcEMA import *
from src.myenv import *
from src.train import *

#pd.set_option('display.max_columns', None)

cols = myenv.float_kline_cols + ['number_of_trades']

label = 'status'
extimator = 'mlp'
stop_loss = 2.0
times_regression_profit_and_loss = 24
#regression_times = times_regression_profit_and_loss
regression_times = 720
regression_features = ['close']
start_train_date = '2010-01-01'
start_test_date = '2023-01-01'

numeric_features = ['close',	'quote_asset_volume',	'number_of_trades',	'taker_buy_quote_asset_volume']
#numeric_features = myenv.float_kline_cols + ['number_of_trades']
date_feature = ['open_time']
all_features = date_feature + numeric_features

strategy = 'SOBE_CAI'


In [2]:
all_data = get_data('BTCUSDT', columns=all_features)
all_data.sort_index(inplace=True)
all_data = all_data[(all_data['open_time'] >= start_train_date)]

all_data = calc_RSI(all_data)
all_data.dropna(inplace=True)
all_data

get_database: name:  /home/marcelo/des/mg_crypto_trader/src/data/BTCUSDT/BTCUSDT.csv
get_database: columns:  ['open_time', 'close', 'quote_asset_volume', 'number_of_trades', 'taker_buy_quote_asset_volume']
get_database: count_rows: 53098 - symbol: BTCUSDT - tail: -1
get_database: duplicated: 0
get_data: Downloading data for symbol: BTCUSDT - max_date: 2023-09-12 21:00:00
get_data: max_date: 2023-09-12 21:00:00 - max_date_aux: 
get_data: Max date database:  2023-09-12 21:00:00
get_klines: shape: (71, 5) - Delta time: 1 seconds
get_data: max_date: 2023-09-14 22:00:00 - max_date_aux: 2023-09-12 21:00:00
get_data: Max date database:  2023-09-14 22:00:00
get_klines: shape: (23, 5) - Delta time: 1 seconds


,open_time,close,quote_asset_volume,number_of_trades,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,
2017-08-17 18:00:00,2017-08-17 18:00:00,4256.970215,2.182594e+05,194,1.916552e+05,BTCUSDT,44.570000
2017-08-17 19:00:00,2017-08-17 19:00:00,4325.229980,1.995219e+05,202,1.794710e+05,BTCUSDT,51.959999
2017-08-17 20:00:00,2017-08-17 20:00:00,4346.740234,2.105489e+05,247,1.843113e+05,BTCUSDT,54.049999
2017-08-17 21:00:00,2017-08-17 21:00:00,4333.549805,2.682849e+05,248,2.221201e+05,BTCUSDT,52.540001
2017-08-17 22:00:00,2017-08-17 22:00:00,4336.799805,2.483683e+05,236,1.918982e+05,BTCUSDT,52.889999
...,...,...,...,...,...,...,...
2023-09-14 18:00:00,2023-09-14 18:00:00,26581.000000,5.316442e+07,-19790,2.707781e+07,BTCUSDT,60.490002
2023-09-14 19:00:00,2023-09-14 19:00:00,26678.140625,2.345936e+07,25099,1.300427e+07,BTCUSDT,64.180000
2023-09-14 20:00:00,2023-09-14 20:00:00,26568.720703,2.376836e+07,23470,1.093311e+07,BTCUSDT,57.650002


In [3]:
features_added = []
if regression_times > 0:
    print('start_train_engine: calculating regresstion_times...')
    all_data, features_added = regresstion_times(all_data, regression_features, regression_times, last_one=False)
    print('start_train_engine: info after calculating regresstion_times: ')
    all_data.info() 
    print('start_train_engine: all_data duplicated: ', all_data.index.duplicated().sum())
numeric_features += features_added

start_train_engine: calculating regresstion_times...
regresstion_times: regression_features:  ['close']
start_train_engine: info after calculating regresstion_times: 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52413 entries, 2017-09-17 00:00:00 to 2023-09-14 22:00:00
Columns: 727 entries, open_time to close_720
dtypes: datetime64[ns](1), float32(724), int16(1), object(1)
memory usage: 146.1+ MB
start_train_engine: all_data duplicated:  0


In [ ]:
#all_data[numeric_features].corr()

In [5]:
all_data = regression_PnL(all_data, label, stop_loss, times_regression_profit_and_loss, strategy=strategy)
all_data

open_time         close  quote_asset_volume  \
ix_open_time                                                                
2017-09-17 00:00:00 2017-09-17 00:00:00   3670.000000        7.607017e+04   
2017-09-17 01:00:00 2017-09-17 01:00:00   3602.000000        9.495384e+04   
2017-09-17 02:00:00 2017-09-17 02:00:00   3640.020020        1.057752e+05   
2017-09-17 03:00:00 2017-09-17 03:00:00   3580.000000        1.793801e+05   
2017-09-17 04:00:00 2017-09-17 04:00:00   3560.010010        5.415628e+04   
...                                 ...           ...                 ...   
2023-09-13 18:00:00 2023-09-13 18:00:00  26111.480469        1.971538e+07   
2023-09-13 19:00:00 2023-09-13 19:00:00  26140.009766        2.280143e+07   
2023-09-13 20:00:00 2023-09-13 20:00:00  26218.039062        1.489744e+07   
2023-09-13 21:00:00 2023-09-13 21:00:00  26226.769531        1.186460e+07   
2023-09-13 22:00:00 2023-09-13 22:00:00  26242.179688        1.285826e+07   

                     number_of_trades  taker_buy_quote_asset_volume   symbol  \
ix_open_time                                                                   
2017-09-17 00:00:00               188                  3.200052e+04  BTCUSDT   
2017-09-17 01:00:00               254                  1.051059e+04  BTCUSDT   
2017-09-17 02:00:00               232                  3.753327e+04  BTCUSDT   
2017-09-17 03:00:00               281                  4.290888e+04  BTCUSDT   
2017-09-17 04:00:00               262                  1.263846e+04  BTCUSDT   
...                               ...                           ...      ...   
2023-09-13 18:00:00             22125                  9.263331e+06  BTCUSDT   
2023-09-13 19:00:00             23661                  1.011956e+07  BTCUSDT   
2023-09-13 20:00:00             19477                  7.460288e+06  BTCUSDT   
2023-09-13 21:00:00             16104                  4.709350e+06  BTCUSDT   
2023-09-13 22:00:00             15814                  5.303040e+06  BTCUSDT   

                           rsi       close_1       close_2       close_3  \
ix_open_time                                                               
2017-09-17 00:00:00  50.720001   3714.949951   3710.000000   3675.000000   
2017-09-17 01:00:00  46.160000   3670.000000   3714.949951   3710.000000   
2017-09-17 02:00:00  48.930000   3602.000000   3670.000000   3714.949951   
2017-09-17 03:00:00  45.000000   3640.020020   3602.000000   3670.000000   
2017-09-17 04:00:00  43.740002   3580.000000   3640.020020   3602.000000   
...                        ...           ...           ...           ...   
2023-09-13 18:00:00  53.580002  26094.130859  26280.050781  26241.310547   
2023-09-13 19:00:00  54.939999  26111.480469  26094.130859  26280.050781   
2023-09-13 20:00:00  58.500000  26140.009766  26111.480469  26094.130859   
2023-09-13 21:00:00  58.900002  26218.039062  26140.009766  26111.480469   
2023-09-13 22:00:00  59.619999  26226.769531  26218.039062  26140.009766   

                          close_4       close_5       close_6       close_7  \
ix_open_time                                                                  
2017-09-17 00:00:00   3662.129883   3664.000000   3681.000000   3617.010010   
2017-09-17 01:00:00   3675.000000   3662.129883   3664.000000   3681.000000   
2017-09-17 02:00:00   3710.000000   3675.000000   3662.129883   3664.000000   
2017-09-17 03:00:00   3714.949951   3710.000000   3675.000000   3662.129883   
2017-09-17 04:00:00   3670.000000   3714.949951   3710.000000   3675.000000   
...                           ...           ...           ...           ...   
2023-09-13 18:00:00  26214.369141  26124.869141  26212.509766  26198.410156   
2023-09-13 19:00:00  26241.310547  26214.369141  26124.869141  26212.509766   
2023-09-13 20:00:00  26280.050781  26241.310547  26214.369141  26124.869141   
2023-09-13 21:00:00  26094.130859  26280.050781  26241.310547  26214.369141   
2023-09-13 22:00:00  26111.480469  26094.130859  2

In [6]:
all_data[all_data.status_sobe != 'ESTAVEL']

open_time         close  quote_asset_volume  \
ix_open_time                                                                
2017-09-17 01:00:00 2017-09-17 01:00:00   3602.000000        9.495384e+04   
2017-09-17 02:00:00 2017-09-17 02:00:00   3640.020020        1.057752e+05   
2017-09-17 03:00:00 2017-09-17 03:00:00   3580.000000        1.793801e+05   
2017-09-17 04:00:00 2017-09-17 04:00:00   3560.010010        5.415628e+04   
2017-09-17 05:00:00 2017-09-17 05:00:00   3557.750000        9.114293e+04   
...                                 ...           ...                 ...   
2023-09-13 07:00:00 2023-09-13 07:00:00  25879.910156        5.670600e+07   
2023-09-13 08:00:00 2023-09-13 08:00:00  25952.880859        1.742835e+07   
2023-09-13 17:00:00 2023-09-13 17:00:00  26094.130859        8.125518e+07   
2023-09-13 18:00:00 2023-09-13 18:00:00  26111.480469        1.971538e+07   
2023-09-13 19:00:00 2023-09-13 19:00:00  26140.009766        2.280143e+07   

                     number_of_trades  taker_buy_quote_asset_volume   symbol  \
ix_open_time                                                                   
2017-09-17 01:00:00               254                  1.051059e+04  BTCUSDT   
2017-09-17 02:00:00               232                  3.753327e+04  BTCUSDT   
2017-09-17 03:00:00               281                  4.290888e+04  BTCUSDT   
2017-09-17 04:00:00               262                  1.263846e+04  BTCUSDT   
2017-09-17 05:00:00               212                  1.360118e+04  BTCUSDT   
...                               ...                           ...      ...   
2023-09-13 07:00:00            -11611                  3.017433e+07  BTCUSDT   
2023-09-13 08:00:00             24448                  8.687982e+06  BTCUSDT   
2023-09-13 17:00:00            -15862                  3.612704e+07  BTCUSDT   
2023-09-13 18:00:00             22125                  9.263331e+06  BTCUSDT   
2023-09-13 19:00:00             23661                  1.011956e+07  BTCUSDT   

                           rsi       close_1       close_2       close_3  \
ix_open_time                                                               
2017-09-17 01:00:00  46.160000   3670.000000   3714.949951   3710.000000   
2017-09-17 02:00:00  48.930000   3602.000000   3670.000000   3714.949951   
2017-09-17 03:00:00  45.000000   3640.020020   3602.000000   3670.000000   
2017-09-17 04:00:00  43.740002   3580.000000   3640.020020   3602.000000   
2017-09-17 05:00:00  43.590000   3560.010010   3580.000000   3640.020020   
...                        ...           ...           ...           ...   
2023-09-13 07:00:00  49.360001  25956.650391  25900.009766  25926.359375   
2023-09-13 08:00:00  52.830002  25879.910156  25956.650391  25900.009766   
2023-09-13 17:00:00  52.779999  26280.050781  26241.310547  26214.369141   
2023-09-13 18:00:00  53.580002  26094.130859  26280.050781  26241.310547   
2023-09-13 19:00:00  54.939999  26111.480469  26094.130859  26280.050781   

                          close_4       close_5       close_6       close_7  \
ix_open_time                                                                  
2017-09-17 01:00:00   3675.000000   3662.129883   3664.000000   3681.000000   
2017-09-17 02:00:00   3710.000000   3675.000000   3662.129883   3664.000000   
2017-09-17 03:00:00   3714.949951   3710.000000   3675.000000   3662.129883   
2017-09-17 04:00:00   3670.000000   3714.949951   3710.000000   3675.000000   
2017-09-17 05:00:00   3602.000000   3670.000000   3714.949951   3710.000000   
...                           ...           ...           ...           ...   
2023-09-13 07:00:00  25879.240234  25910.500000  25957.470703  25919.509766   
2023-09-13 08:00:00  25926.359375  25879.240234  25910.500000  25957.470703   
2023-09-13 17:00:00  26124.869141  26212.509766  26198.410156  26151.900391   
2023-09-13 18:00:00  26214.369141  26124.869141  26212.509766  26198.410156   
2023-09-13 19:00:00  26241.310547  26214.369141  2

In [7]:
all_data[all_data.status_cai != 'ESTAVEL']

open_time         close  quote_asset_volume  \
ix_open_time                                                                
2017-09-17 00:00:00 2017-09-17 00:00:00   3670.000000        7.607017e+04   
2017-09-17 01:00:00 2017-09-17 01:00:00   3602.000000        9.495384e+04   
2017-09-17 02:00:00 2017-09-17 02:00:00   3640.020020        1.057752e+05   
2017-09-17 03:00:00 2017-09-17 03:00:00   3580.000000        1.793801e+05   
2017-09-17 10:00:00 2017-09-17 10:00:00   3577.439941        9.224545e+04   
...                                 ...           ...                 ...   
2023-09-11 09:00:00 2023-09-11 09:00:00  25736.490234        4.477737e+07   
2023-09-11 10:00:00 2023-09-11 10:00:00  25711.519531        6.007401e+07   
2023-09-11 11:00:00 2023-09-11 11:00:00  25601.269531        2.753041e+07   
2023-09-11 12:00:00 2023-09-11 12:00:00  25718.000000        3.027743e+07   
2023-09-11 13:00:00 2023-09-11 13:00:00  25617.500000        3.192405e+07   

                     number_of_trades  taker_buy_quote_asset_volume   symbol  \
ix_open_time                                                                   
2017-09-17 00:00:00               188                  3.200052e+04  BTCUSDT   
2017-09-17 01:00:00               254                  1.051059e+04  BTCUSDT   
2017-09-17 02:00:00               232                  3.753327e+04  BTCUSDT   
2017-09-17 03:00:00               281                  4.290888e+04  BTCUSDT   
2017-09-17 10:00:00               364                  6.562364e+04  BTCUSDT   
...                               ...                           ...      ...   
2023-09-11 09:00:00            -30232                  2.036595e+07  BTCUSDT   
2023-09-11 10:00:00            -17587                  2.825829e+07  BTCUSDT   
2023-09-11 11:00:00             26250                  1.102782e+07  BTCUSDT   
2023-09-11 12:00:00             30020                  1.618227e+07  BTCUSDT   
2023-09-11 13:00:00             30672                  1.457989e+07  BTCUSDT   

                           rsi       close_1       close_2       close_3  \
ix_open_time                                                               
2017-09-17 00:00:00  50.720001   3714.949951   3710.000000   3675.000000   
2017-09-17 01:00:00  46.160000   3670.000000   3714.949951   3710.000000   
2017-09-17 02:00:00  48.930000   3602.000000   3670.000000   3714.949951   
2017-09-17 03:00:00  45.000000   3640.020020   3602.000000   3670.000000   
2017-09-17 10:00:00  46.450001   3549.979980   3535.010010   3559.000000   
...                        ...           ...           ...           ...   
2023-09-11 09:00:00  43.639999  25805.259766  25813.650391  25839.550781   
2023-09-11 10:00:00  41.660000  25736.490234  25805.259766  25813.650391   
2023-09-11 11:00:00  34.270000  25711.519531  25736.490234  25805.259766   
2023-09-11 12:00:00  45.330002  25601.269531  25711.519531  25736.490234   
2023-09-11 13:00:00  39.209999  25718.000000  25601.269531  25711.519531   

                          close_4       close_5       close_6       close_7  \
ix_open_time                                                                  
2017-09-17 00:00:00   3662.129883   3664.000000   3681.000000   3617.010010   
2017-09-17 01:00:00   3675.000000   3662.129883   3664.000000   3681.000000   
2017-09-17 02:00:00   3710.000000   3675.000000   3662.129883   3664.000000   
2017-09-17 03:00:00   3714.949951   3710.000000   3675.000000   3662.129883   
2017-09-17 10:00:00   3563.050049   3557.750000   3560.010010   3580.000000   
...                           ...           ...           ...           ...   
2023-09-11 09:00:00  25892.970703  25778.070312  25740.390625  25735.720703   
2023-09-11 10:00:00  25839.550781  25892.970703  25778.070312  25740.390625   
2023-09-11 11:00:00  25813.650391  25839.550781  25892.970703  25778.070312   
2023-09-11 12:00:00  25805.259766  25813.650391  25839.550781  25892.970703   
2023-09-11 13:00:00  25736.490234  25805.259766  2

In [8]:
"""
all_data.status_cai = all_data.status_cai.astype(str)
all_data.status_sobe = all_data.status_sobe.astype(str)
all_data.loc[(all_data.status_cai == 'CAI_2.0') & (all_data.status_sobe == 'SOBE_2.0'), 'status_sobe'] = 'SOBE_2.0'
all_data.loc[(all_data.status_cai == 'CAI_2.0') & (all_data.status_sobe == 'SOBE_2.0'), 'status_cai'] = 'ESTAVEL'
all_data[(all_data.status_cai == 'CAI_2.0') & (all_data.status_sobe == 'SOBE_2.0')]
"""

"\nall_data.status_cai = all_data.status_cai.astype(str)\nall_data.status_sobe = all_data.status_sobe.astype(str)\nall_data.loc[(all_data.status_cai == 'CAI_2.0') & (all_data.status_sobe == 'SOBE_2.0'), 'status_sobe'] = 'SOBE_2.0'\nall_data.loc[(all_data.status_cai == 'CAI_2.0') & (all_data.status_sobe == 'SOBE_2.0'), 'status_cai'] = 'ESTAVEL'\nall_data[(all_data.status_cai == 'CAI_2.0') & (all_data.status_sobe == 'SOBE_2.0')]\n"

In [9]:
all_data

open_time         close  quote_asset_volume  \
ix_open_time                                                                
2017-09-17 00:00:00 2017-09-17 00:00:00   3670.000000        7.607017e+04   
2017-09-17 01:00:00 2017-09-17 01:00:00   3602.000000        9.495384e+04   
2017-09-17 02:00:00 2017-09-17 02:00:00   3640.020020        1.057752e+05   
2017-09-17 03:00:00 2017-09-17 03:00:00   3580.000000        1.793801e+05   
2017-09-17 04:00:00 2017-09-17 04:00:00   3560.010010        5.415628e+04   
...                                 ...           ...                 ...   
2023-09-13 18:00:00 2023-09-13 18:00:00  26111.480469        1.971538e+07   
2023-09-13 19:00:00 2023-09-13 19:00:00  26140.009766        2.280143e+07   
2023-09-13 20:00:00 2023-09-13 20:00:00  26218.039062        1.489744e+07   
2023-09-13 21:00:00 2023-09-13 21:00:00  26226.769531        1.186460e+07   
2023-09-13 22:00:00 2023-09-13 22:00:00  26242.179688        1.285826e+07   

                     number_of_trades  taker_buy_quote_asset_volume   symbol  \
ix_open_time                                                                   
2017-09-17 00:00:00               188                  3.200052e+04  BTCUSDT   
2017-09-17 01:00:00               254                  1.051059e+04  BTCUSDT   
2017-09-17 02:00:00               232                  3.753327e+04  BTCUSDT   
2017-09-17 03:00:00               281                  4.290888e+04  BTCUSDT   
2017-09-17 04:00:00               262                  1.263846e+04  BTCUSDT   
...                               ...                           ...      ...   
2023-09-13 18:00:00             22125                  9.263331e+06  BTCUSDT   
2023-09-13 19:00:00             23661                  1.011956e+07  BTCUSDT   
2023-09-13 20:00:00             19477                  7.460288e+06  BTCUSDT   
2023-09-13 21:00:00             16104                  4.709350e+06  BTCUSDT   
2023-09-13 22:00:00             15814                  5.303040e+06  BTCUSDT   

                           rsi       close_1       close_2       close_3  \
ix_open_time                                                               
2017-09-17 00:00:00  50.720001   3714.949951   3710.000000   3675.000000   
2017-09-17 01:00:00  46.160000   3670.000000   3714.949951   3710.000000   
2017-09-17 02:00:00  48.930000   3602.000000   3670.000000   3714.949951   
2017-09-17 03:00:00  45.000000   3640.020020   3602.000000   3670.000000   
2017-09-17 04:00:00  43.740002   3580.000000   3640.020020   3602.000000   
...                        ...           ...           ...           ...   
2023-09-13 18:00:00  53.580002  26094.130859  26280.050781  26241.310547   
2023-09-13 19:00:00  54.939999  26111.480469  26094.130859  26280.050781   
2023-09-13 20:00:00  58.500000  26140.009766  26111.480469  26094.130859   
2023-09-13 21:00:00  58.900002  26218.039062  26140.009766  26111.480469   
2023-09-13 22:00:00  59.619999  26226.769531  26218.039062  26140.009766   

                          close_4       close_5       close_6       close_7  \
ix_open_time                                                                  
2017-09-17 00:00:00   3662.129883   3664.000000   3681.000000   3617.010010   
2017-09-17 01:00:00   3675.000000   3662.129883   3664.000000   3681.000000   
2017-09-17 02:00:00   3710.000000   3675.000000   3662.129883   3664.000000   
2017-09-17 03:00:00   3714.949951   3710.000000   3675.000000   3662.129883   
2017-09-17 04:00:00   3670.000000   3714.949951   3710.000000   3675.000000   
...                           ...           ...           ...           ...   
2023-09-13 18:00:00  26214.369141  26124.869141  26212.509766  26198.410156   
2023-09-13 19:00:00  26241.310547  26214.369141  26124.869141  26212.509766   
2023-09-13 20:00:00  26280.050781  26241.310547  26214.369141  26124.869141   
2023-09-13 21:00:00  26094.130859  26280.050781  26241.310547  26214.369141   
2023-09-13 22:00:00  26111.480469  26094.130859  2

In [10]:
all_data['open_time'] = all_data.index
train_data = all_data[(all_data['open_time'] < start_test_date)]
test_data = all_data[(all_data['open_time'] >= start_test_date)]

In [11]:
#sobe_data.reset_index(drop=True, inplace=True)
#sobe_data.drop(columns=['open_time'], inplace=True)
#sobe_data

#cai_data.reset_index(drop=True, inplace=True)
#cai_data.drop(columns=['open_time'], inplace=True)
#cai_data

In [12]:
print(f'numeric_features: {numeric_features}')
sobe_exp = ClassificationExperiment()
cai_exp = ClassificationExperiment()

sobe_setup = sobe_exp.setup(train_data[date_feature + numeric_features + ['rsi', label + '_sobe']].reset_index(drop=True),
                        train_size=0.7,
                        target=label + '_sobe',
                        numeric_features=numeric_features + ['rsi'],
                        date_features=['open_time'],
                        create_date_columns=["hour", "day", "month"],
                        fold_strategy='timeseries',
                        fold=3,
                        session_id=123,
                        normalize=True,
                        use_gpu=False,
                        verbose=True,
                        n_jobs=20,
                        log_experiment=False)
cai_setup = cai_exp.setup(train_data[date_feature + numeric_features + ['rsi', label + '_cai']].reset_index(drop=True),
                        train_size=0.7,
                        target=label + '_cai',
                        numeric_features=numeric_features + ['rsi'],
                        date_features=['open_time'],
                        create_date_columns=["hour", "day", "month"],
                        fold_strategy='timeseries',
                        fold=3,
                        session_id=123,
                        normalize=True,
                        use_gpu=False,
                        verbose=True,
                        n_jobs=20,
                        log_experiment=False)


numeric_features: ['close', 'quote_asset_volume', 'number_of_trades', 'taker_buy_quote_asset_volume', 'close_1', 'close_2', 'close_3', 'close_4', 'close_5', 'close_6', 'close_7', 'close_8', 'close_9', 'close_10', 'close_11', 'close_12', 'close_13', 'close_14', 'close_15', 'close_16', 'close_17', 'close_18', 'close_19', 'close_20', 'close_21', 'close_22', 'close_23', 'close_24', 'close_25', 'close_26', 'close_27', 'close_28', 'close_29', 'close_30', 'close_31', 'close_32', 'close_33', 'close_34', 'close_35', 'close_36', 'close_37', 'close_38', 'close_39', 'close_40', 'close_41', 'close_42', 'close_43', 'close_44', 'close_45', 'close_46', 'close_47', 'close_48', 'close_49', 'close_50', 'close_51', 'close_52', 'close_53', 'close_54', 'close_55', 'close_56', 'close_57', 'close_58', 'close_59', 'close_60', 'close_61', 'close_62', 'close_63', 'close_64', 'close_65', 'close_66', 'close_67', 'close_68', 'close_69', 'close_70', 'close_71', 'close_72', 'close_73', 'close_74', 'close_75', 'close_

,Description,Value
0,Session id,123
1,Target,status_sobe
2,Target type,Binary
3,Target mapping,"ESTAVEL: 0, SOBE_2.0: 1"
4,Original data shape,"(46247, 727)"
5,Transformed data shape,"(46247, 729)"
6,Transformed train set shape,"(32372, 729)"
7,Transformed test set shape,"(13875, 729)"
8,Numeric features,725
9,Date features,1


,Description,Value
0,Session id,123
1,Target,status_cai
2,Target type,Binary
3,Target mapping,"CAI_2.0: 0, ESTAVEL: 1"
4,Original data shape,"(46247, 727)"
5,Transformed data shape,"(46247, 729)"
6,Transformed train set shape,"(32372, 729)"
7,Transformed test set shape,"(13875, 729)"
8,Numeric features,725
9,Date features,1


In [13]:
sobe_best = sobe_setup.create_model(estimator=extimator)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6149,0.6197,0.3160,0.4960,0.3861,0.1251,0.1319
1,0.6558,0.6590,0.3742,0.5921,0.4586,0.2246,0.2373
2,0.6739,0.6909,0.3573,0.6283,0.4555,0.2471,0.2672
Mean,0.6482,0.6565,0.3492,0.5721,0.4334,0.1990,0.2121
Std,0.0247,0.0291,0.0244,0.0559,0.0335,0.0530,0.0581


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
cai_best = cai_setup.create_model(estimator=extimator)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6467,0.6437,0.8359,0.6757,0.7473,0.1817,0.1935
1,0.6745,0.6658,0.9361,0.6772,0.7859,0.1750,0.2227
2,0.7036,0.7426,0.8911,0.7103,0.7905,0.3051,0.3291
Mean,0.6749,0.6841,0.8877,0.6878,0.7746,0.2206,0.2484
Std,0.0232,0.0424,0.0410,0.0160,0.0194,0.0598,0.0583


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
_test_data = test_data.drop(columns=label + '_sobe')

df_final_predict, _ = validate_score_test_data(sobe_exp, sobe_best, label + '_sobe', test_data, _test_data)

start_test_date = df_final_predict['open_time'].min()
end_test_date = df_final_predict['open_time'].max()

print('Min Data: ', start_test_date)
print('Max Data: ', end_test_date)
simule_trading_crypto(df_final_predict, start_test_date, end_test_date, 100, stop_loss, False)

start_train_engine: predicting final model...


Score Mean: 0.6436014327580593
Score Group:                _score
status_sobe          
ESTAVEL      0.699273
SOBE_2.0     0.441598
Min Data:  2023-01-01 00:00:00
Max Data:  2023-09-13 22:00:00
[1][SOBE_2.0][2023-01-01 01h] => Compra: 16551.4707
[89][SOBE_2.0][2023-01-04 17h] => Venda: 16893.2402 => Diff: 2.06% ==> PnL: $ 102.06
[90][SOBE_2.0][2023-01-04 18h] => Compra: 16948.0391
[208][SOBE_2.0][2023-01-09 16h] => Venda: 17327.8398 => Diff: 2.24% ==> PnL: $ 104.35
[209][SOBE_2.0][2023-01-09 17h] => Compra: 17356.9609
[263][SOBE_2.0][2023-01-11 23h] => Venda: 17943.2598 => Diff: 3.38% ==> PnL: $ 107.87
[264][SOBE_2.0][2023-01-12 00h] => Compra: 18264.0996
[281][SOBE_2.0][2023-01-12 17h] => Venda: 18837.0605 => Diff: 3.14% ==> PnL: $ 111.25
[282][SOBE_2.0][2023-01-12 18h] => Compra: 18796.9297
[303][SOBE_2.0][2023-01-13 15h] => Venda: 19261.1895 => Diff: 2.47% ==> PnL: $ 114.00
[304][SOBE_2.0][2023-01-13 16h] => Compra: 19133.3906
[309][SOBE_2.0][2023-01-13 21h] => Venda: 19816.8105 => 

136.28000000000006

In [16]:
_test_data = test_data.drop(columns=label + '_cai')

df_final_predict, _ = validate_score_test_data(cai_exp, cai_best, label + '_cai', test_data, _test_data)

start_test_date = df_final_predict['open_time'].min()
end_test_date = df_final_predict['open_time'].max()

print('Min Data: ', start_test_date)
print('Max Data: ', end_test_date)
simule_trading_crypto(df_final_predict, start_test_date, end_test_date, 100, stop_loss, False)

start_train_engine: predicting final model...


Score Mean: 0.7097036795831977
Score Group:               _score
status_cai          
CAI_2.0     0.185603
ESTAVEL     0.830828
Min Data:  2023-01-01 00:00:00
Max Data:  2023-09-13 22:00:00
[1][CAI_2.0][2023-01-01 01h] => Compra: 16551.4707
[89][CAI_2.0][2023-01-04 17h] => Venda: 16893.2402 => Diff: 2.06% ==> PnL: $ 97.94
[92][CAI_2.0][2023-01-04 20h] => Compra: 16806.4902
[192][CAI_2.0][2023-01-09 00h] => Venda: 17184.5098 => Diff: 2.25% ==> PnL: $ 95.74
[195][CAI_2.0][2023-01-09 03h] => Compra: 17197.0000
[259][CAI_2.0][2023-01-11 19h] => Venda: 17550.2891 => Diff: 2.05% ==> PnL: $ 93.77
[261][CAI_2.0][2023-01-11 21h] => Compra: 17558.5996
[263][CAI_2.0][2023-01-11 23h] => Venda: 17943.2598 => Diff: 2.19% ==> PnL: $ 91.72
[271][CAI_2.0][2023-01-12 07h] => Compra: 18138.2207
[281][CAI_2.0][2023-01-12 17h] => Venda: 18837.0605 => Diff: 3.85% ==> PnL: $ 88.19
[286][CAI_2.0][2023-01-12 22h] => Compra: 18913.9199
[306][CAI_2.0][2023-01-13 18h] => Venda: 19292.4395 => Diff: 2.00% ==> PnL: 

80.47999999999996

In [17]:
sobe_tuned_model = sobe_exp.tune_model(sobe_best)
sobe_tuned_model

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6798,0.7258,0.4792,0.6036,0.5342,0.2952,0.2998
1,0.7374,0.7865,0.4814,0.7560,0.5883,0.4091,0.4316
2,0.7581,0.8163,0.5405,0.7563,0.6304,0.4579,0.4722
Mean,0.7251,0.7762,0.5004,0.7053,0.5843,0.3874,0.4012
Std,0.0331,0.0377,0.0284,0.0719,0.0394,0.0682,0.0736


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


MLPClassifier(activation='logistic', alpha=1e-07, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=[100, 50], learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=123, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [18]:
cai_tuned_model = cai_exp.tune_model(cai_best)
cai_tuned_model

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7099,0.7485,0.8849,0.7171,0.7922,0.3287,0.3493
1,0.7414,0.7841,0.8644,0.7622,0.8101,0.4098,0.4179
2,0.7621,0.8207,0.9199,0.7548,0.8292,0.4498,0.4744
Mean,0.7378,0.7845,0.8898,0.7447,0.8105,0.3961,0.4139
Std,0.0215,0.0295,0.0229,0.0197,0.0151,0.0504,0.0511


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


MLPClassifier(activation='logistic', alpha=1e-07, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=[100, 50], learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=123, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [19]:
_test_data = test_data.drop(columns=label + '_sobe')

df_final_predict, _ = validate_score_test_data(sobe_exp, sobe_tuned_model, label + '_sobe', test_data, _test_data)

start_test_date = df_final_predict['open_time'].min()
end_test_date = df_final_predict['open_time'].max()

print('Min Data: ', start_test_date)
print('Max Data: ', end_test_date)
simule_trading_crypto(df_final_predict, start_test_date, end_test_date, 100, stop_loss, False)

start_train_engine: predicting final model...


Score Mean: 0.4715076522305438
Score Group:                _score
status_sobe          
ESTAVEL      0.465421
SOBE_2.0     0.493595
Min Data:  2023-01-01 00:00:00
Max Data:  2023-09-13 22:00:00
[1][SOBE_2.0][2023-01-01 01h] => Compra: 16551.4707
[89][SOBE_2.0][2023-01-04 17h] => Venda: 16893.2402 => Diff: 2.06% ==> PnL: $ 102.06
[90][SOBE_2.0][2023-01-04 18h] => Compra: 16948.0391
[208][SOBE_2.0][2023-01-09 16h] => Venda: 17327.8398 => Diff: 2.24% ==> PnL: $ 104.35
[209][SOBE_2.0][2023-01-09 17h] => Compra: 17356.9609
[263][SOBE_2.0][2023-01-11 23h] => Venda: 17943.2598 => Diff: 3.38% ==> PnL: $ 107.87
[264][SOBE_2.0][2023-01-12 00h] => Compra: 18264.0996
[281][SOBE_2.0][2023-01-12 17h] => Venda: 18837.0605 => Diff: 3.14% ==> PnL: $ 111.25
[283][SOBE_2.0][2023-01-12 19h] => Compra: 18885.3496
[306][SOBE_2.0][2023-01-13 18h] => Venda: 19292.4395 => Diff: 2.16% ==> PnL: $ 113.65
[307][SOBE_2.0][2023-01-13 19h] => Compra: 19362.0098
[309][SOBE_2.0][2023-01-13 21h] => Venda: 19816.8105 => 

142.13000000000002

In [20]:
_test_data = test_data.drop(columns=label + '_cai')

df_final_predict, _ = validate_score_test_data(cai_exp, cai_tuned_model, label + '_cai', test_data, _test_data)

start_test_date = df_final_predict['open_time'].min()
end_test_date = df_final_predict['open_time'].max()

print('Min Data: ', start_test_date)
print('Max Data: ', end_test_date)
simule_trading_crypto(df_final_predict, start_test_date, end_test_date, 100, stop_loss, False)

start_train_engine: predicting final model...


Score Mean: 0.39384565288179746
Score Group:               _score
status_cai          
CAI_2.0     0.702515
ESTAVEL     0.322510
Min Data:  2023-01-01 00:00:00
Max Data:  2023-09-13 22:00:00
[1][CAI_2.0][2023-01-01 01h] => Compra: 16551.4707
[89][CAI_2.0][2023-01-04 17h] => Venda: 16893.2402 => Diff: 2.06% ==> PnL: $ 97.94
[90][CAI_2.0][2023-01-04 18h] => Compra: 16948.0391
[208][CAI_2.0][2023-01-09 16h] => Venda: 17327.8398 => Diff: 2.24% ==> PnL: $ 95.75
[209][CAI_2.0][2023-01-09 17h] => Compra: 17356.9609
[263][CAI_2.0][2023-01-11 23h] => Venda: 17943.2598 => Diff: 3.38% ==> PnL: $ 92.52
[264][CAI_2.0][2023-01-12 00h] => Compra: 18264.0996
[281][CAI_2.0][2023-01-12 17h] => Venda: 18837.0605 => Diff: 3.14% ==> PnL: $ 89.62
[282][CAI_2.0][2023-01-12 18h] => Compra: 18796.9297
[303][CAI_2.0][2023-01-13 15h] => Venda: 19261.1895 => Diff: 2.47% ==> PnL: $ 87.41
[304][CAI_2.0][2023-01-13 16h] => Compra: 19133.3906
[309][CAI_2.0][2023-01-13 21h] => Venda: 19816.8105 => Diff: 3.57% ==> PnL:

68.56999999999998

In [21]:
sobe_final_model = sobe_exp.finalize_model(sobe_tuned_model)
sobe_final_model

In [ ]:
cai_final_model = cai_exp.finalize_model(cai_tuned_model)
cai_final_model

In [ ]:
_test_data = test_data.drop(columns=label + '_sobe')

df_final_predict, sobe_res_score = validate_score_test_data(sobe_exp, sobe_final_model, label + '_sobe', test_data, _test_data)

start_test_date = df_final_predict['open_time'].min()
end_test_date = df_final_predict['open_time'].max()

print('Min Data: ', start_test_date)
print('Max Data: ', end_test_date)
simule_trading_crypto(df_final_predict, start_test_date, end_test_date, 100, stop_loss, False)

In [ ]:
_test_data = test_data.drop(columns=label + '_cai')

df_final_predict, cai_res_score = validate_score_test_data(cai_exp, cai_final_model, label + '_cai', test_data, _test_data)

start_test_date = df_final_predict['open_time'].min()
end_test_date = df_final_predict['open_time'].max()

print('Min Data: ', start_test_date)
print('Max Data: ', end_test_date)
simule_trading_crypto(df_final_predict, start_test_date, end_test_date, 100, stop_loss, False)